In [1]:
import pandas as pd

In [2]:
test = pd.read_csv('test.csv')

In [3]:
sample = pd.read_csv('sample.csv')

In [4]:
train = pd.read_csv('train.csv')

In [5]:
print("전체 행 개수 :", len(train))
print(train.isna().sum())     

전체 행 개수 : 1200000
id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64


## 데이터 전처리(나머지 변수, 공통)

In [36]:
# Number of Dependents: 구간화(Binning) & 결측 플래그 추가

# 결측 플래그 생성
train['Number_of_Dependents_missing'] = train['Number of Dependents'].isnull().astype(int)

# 구간화 예시 (0명, 1~2명, 3명 이상)
bins = [-np.inf, 0, 2, np.inf]
labels = ['0', '1-2', '3+']
train['Number_of_Dependents_bin'] = pd.cut(train['Number of Dependents'], bins=bins, labels=labels)

In [37]:
# 나이에 대한 결측 플래그 및 중앙값 대체 예시
train['Age_missing'] = train['Age'].isnull().astype(int)
train['Age'] = train['Age'].fillna(train['Age'].median())

# Annual Income
train['Annual_Income_missing'] = train['Annual Income'].isnull().astype(int)
train['Annual Income'] = train['Annual Income'].fillna(train['Annual Income'].median())

# Health Score
train['Health_Score_missing'] = train['Health Score'].isnull().astype(int)
train['Health Score'] = train['Health Score'].fillna(train['Health Score'].median())

# Credit Score
train['Credit_Score_missing'] = train['Credit Score'].isnull().astype(int)
train['Credit Score'] = train['Credit Score'].fillna('Missing')

# Customer Feedback
train['Customer_Feedback_missing'] = train['Customer Feedback'].isnull().astype(int)
train['Customer Feedback'] = train['Customer Feedback'].fillna('No Feedback')

In [38]:
# Vehicle Age 결측 행 삭제
train = train[train['Vehicle Age'].notnull()]

In [39]:
# Insurance Duration 결측 행 삭제
train = train[train['Insurance Duration'].notnull()]

In [40]:
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Exercise Frequency,Property Type,Premium Amount,Number_of_Dependents_missing,Number_of_Dependents_bin,Age_missing,Annual_Income_missing,Health_Score_missing,Credit_Score_missing,Customer_Feedback_missing
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,Weekly,House,2869.0,0,1-2,0,0,0,0,0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,Monthly,House,1483.0,0,3+,0,0,0,0,0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,Weekly,House,567.0,0,3+,0,0,0,0,0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,Daily,Apartment,765.0,0,1-2,0,0,0,0,0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,Weekly,House,2022.0,0,1-2,0,0,0,0,0


### 상관계수

In [41]:
!pip install seaborn matplotlib


zsh:1: /Users/t2023-m0149/Documents/GitHub/4gun4_new_project/alex/bin/pip: bad interpreter: /Users/t2023-m0149/Documents/GitHub/4gun4_new_project-1/alex/bin/python3: no such file or directory


In [35]:
# 1) 수치형-수치형: 피어슨 상관계수 + 히트맵
import seaborn as sns
import matplotlib.pyplot as plt

num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Insurance Duration']
corr = train1[num_cols].corr(method='pearson')
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', square=True)
plt.title('Pearson Correlation: Numeric Features')
plt.show()


ModuleNotFoundError: No module named 'seaborn'

# 실험1

1. Occupation
- 1-1. 'unknown'으로 대체
- 1-2. 범주별 annual income값 비교해서 대체

2. Previous Claims
- 2-1. 0값으로 처리
- 2-2. 0~9 범주화, null로 대체

### 실험1 > 1-1 & 2-1

In [19]:
train1 = train.copy()
train1['Occupation'] = train1['Occupation'].fillna('Unknown')
train1['Previous Claims'] = train1['Previous Claims'].fillna(0)